In [1]:
import numpy as np


In [2]:

# Define transition probabilities, rewards, and discount factor
num_states = 16  # Assuming a 4x4 grid
num_actions = 4  # Up, Down, Left, Right

# Initialize transition probabilities
transition_probs = np.zeros((num_states, num_actions, num_states))

# Fill in transition probabilities based on the grid world dynamics
for s in range(num_states):
    for a in range(num_actions):
        if a == 0:  # Up
            next_state = max(0, s - 4)  # Assuming a 4x4 grid
        elif a == 1:  # Down
            next_state = min(15, s + 4)  # Assuming a 4x4 grid
        elif a == 2:  # Left
            next_state = max(0, s - 1) if s % 4 != 0 else s  # Assuming a 4x4 grid
        else:  # Right
            next_state = min(15, s + 1) if (s + 1) % 4 != 0 else s  # Assuming a 4x4 grid

        transition_probs[s][a][next_state] = 1.0  # Deterministic transitions




In [3]:
# Define rewards for each state
rewards = np.zeros(num_states)
rewards[15] = 1  # Setting the reward for the terminal state

# Discount factor
discount_factor = 0.9  # Adjust as needed

# Policy Iteration Algorithm
def policy_evaluation(policy, num_states, num_actions, transition_probs, rewards, discount_factor, epsilon=1e-6):
    V = np.zeros(num_states)
    while True:
        delta = 0
        for s in range(num_states):
            v = V[s]
            action = np.argmax(policy[s])  # Choose action according to the policy
            value = 0
            for s_prime in range(num_states):
                value += transition_probs[s][action][s_prime] * (rewards[s] + discount_factor * V[s_prime])
            V[s] = value
            delta = max(delta, abs(v - V[s]))
        if delta < epsilon:
            break
    return V

def policy_improvement(V, num_states, num_actions, transition_probs, rewards, discount_factor):
    policy_stable = True
    policy = np.zeros((num_states, num_actions))
    for s in range(num_states):
        old_action = np.argmax(policy[s])
        action_values = np.zeros(num_actions)
        for a in range(num_actions):
            for s_prime in range(num_states):
                action_values[a] += transition_probs[s][a][s_prime] * (rewards[s] + discount_factor * V[s_prime])
        best_action = np.argmax(action_values)
        if old_action != best_action:
            policy_stable = False
        policy[s][best_action] = 1.0

    return policy, policy_stable

def policy_iteration(num_states, num_actions, transition_probs, rewards, discount_factor):
    policy = np.ones((num_states, num_actions)) / num_actions  # Initialize with a uniform policy

    while True:
        V = policy_evaluation(policy, num_states, num_actions, transition_probs, rewards, discount_factor)
        new_policy, policy_stable = policy_improvement(V, num_states, num_actions, transition_probs, rewards, discount_factor)
        if policy_stable:
            break
        policy = new_policy

    return policy, V


In [ ]:
# Usage
optimal_policy, optimal_value_function = policy_iteration(num_states, num_actions, transition_probs, rewards, discount_factor)

print("Optimal Policy:")
print(optimal_policy)
print("\nOptimal Value Function:")
print(optimal_value_function)
